In [2]:
from pathlib import Path
import csv
import pandas as pd

In [3]:
# loading the data

notebook_dir = Path.cwd()
# the data is located in data/raw folder
data_dir = notebook_dir.parent.absolute() / 'data_raw'
csv_dir = notebook_dir.parent.absolute() / 'clean_csvs'

#convert txt of imessage texts to string
path_raw = data_dir / 'aarush-imessage.txt'
data = open(path_raw, 'r').read()
path_csv_grouped = csv_dir / 'messages_grouped_by_user' / 'aarush-imessage-grouped.csv'
path_csv_separate = csv_dir / 'messages_separate' / 'aarush-imessage-separate.csv'


In [4]:
imessage = ''
for line in data.splitlines():
    if line[:4] == '    ':
        continue
    if line.strip() == '':
        continue
    imessage += line.strip() + '\n'



In [5]:
filename = "output.txt"

# Open a file in write mode ('w') and write the string to it
with open(filename, 'w') as file:
    file.write(imessage)

print(f"String has been written to {filename}")

String has been written to output.txt


In [6]:
def getDate(messageLine):
    date = messageLine.splitlines()[0]
    if date.find('PM') != -1: #sent in the evening
        endInd = date.find('PM') + 2
    else:
        endInd = date.find('AM') + 2
    date = date[:endInd]
    return date

In [7]:
def getWordCount(messageLine):
    messageLine = messageLine.strip()
    return(len(messageLine.split(' ')))

In [8]:
months = {'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}

In [9]:
reactions = None
game_pigeon = False
text = ''
num_sep_texts = 0
def parseImessageTxt(imessage):
    #split imessage string based on \n 
    #analyze each message block separately
    message_dict = []
    imessage = imessage.replace('This message responded to an earlier message.', '')
    res = {}
    reaction = False
    gamePigeon = False
    date = ''
    #columns: ['date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'game_pigeon']
    for message in imessage.splitlines():
        if message.isspace():
            continue
        message = message.strip()
        if message[:6] == 'Edited':
            continue
        if gamePigeon:
            if message[:3] in months:
                gamePigeon = False
            else:
                continue
        if reaction:
            reaction_message = (message.split(' ')[0])
            reactions = reaction_message
            reaction = False
            continue
        if message[:3] in months and message[3] == ' ':
            if date:
                res = {'date': date, 'user': user, 'text' : text, 'word count' : getWordCount(text), 
                   'number of separated texts': num_sep_texts, 'reactions' : reactions, 
                   'game pigeon': game_pigeon}
                message_dict.append(res)
            res, text = {}, ''
            reactions = None
            game_pigeon = False
            text = ''
            num_sep_texts = 0
            date = getDate(message)
        elif message.find('GamePigeon message') != -1:
            game_pigeon = True
            gamePigeon = True
            continue

        elif message == 'Me':
            user = 'Samyukta'
        elif message == '+16692928203':
            user = 'Aarush'
        elif message.find('(Read by') != -1 or message.find('Reactions') != -1:
            if message.find('(Read by') != -1:
                firstIndex = message.find('(Read by')
                message = message[:firstIndex - 1]
            if message.find('Reactions:') != -1:
                reaction = True
                continue
        else:
            text = text + message
            num_sep_texts += 1
    res = {'date': date, 'user': user, 'text' : text, 'word count' : getWordCount(text), 
                   'number of separated texts': num_sep_texts, 'reactions' : reactions, 
                   'game pigeon': game_pigeon}
    message_dict.append(res)
    #message_dict.pop(0)
    return message_dict


In [10]:
print(text)

In [11]:
test = '''Dec 27, 2023  3:50:35 PM (Read by you after 2 minutes, 41 seconds)
+16692928203
why r u imessage
Dec 27, 2023  3:50:38 PM (Read by you after 2 minutes, 38 seconds)
+16692928203
r u straight now
Dec 27, 2023  3:53:24 PM (Read by them after 1 second)
Me
Ugh I don’t know
This message responded to an earlier message.'''

message_dict = parseImessageTxt(imessage)


In [12]:
#save dictionary as csv
aarushImsgCSV = open(path_csv_separate, 'w')
writer = csv.writer(aarushImsgCSV)
writer.writerow(['date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'game_pigeon'])
for dictionary in message_dict:
    writer.writerow(dictionary.values())
aarushImsgCSV.close()
aarushImsgCSV = open(path_csv_separate, 'r')
print("The content of the csv file is:")
print(aarushImsgCSV.read())
aarushImsgCSV.close()

The content of the csv file is:
date,user,text,word count,number of separated texts,reactions,game_pigeon

"Dec 27, 2023  3:30:14 PM",Samyukta,Hey girl! Big fan of your writing Xx- Collin Hoover,10,1,,False

"Dec 27, 2023  3:30:36 PM",Aarush,stop,1,1,,False

"Dec 27, 2023  3:30:39 PM",Aarush,send me location,3,1,,False

"Dec 27, 2023  3:34:07 PM",Samyukta,Why,1,1,,False

"Dec 27, 2023  3:34:15 PM",Samyukta,girl get on snapchat if u want to see locations,10,1,,False

"Dec 27, 2023  3:34:38 PM",Aarush,can u stop being my opp,6,1,,False

"Dec 27, 2023  3:34:52 PM",Aarush,attachments/11/dea07bac-b0ff-4db7-b916-2dfd96f1cc56.heic,1,1,,False

"Dec 27, 2023  3:47:28 PM",Samyukta,Get off iMessage and get on snapchat boy!!,8,2,,False

"Dec 27, 2023  3:47:41 PM",Samyukta,Snap baddies waiting for ya 😜,6,1,,False

"Dec 27, 2023  3:47:48 PM",Samyukta,ZAMN,1,2,,False

"Dec 27, 2023  3:47:54 PM",Aarush,what the h*ck,3,1,,False

"Dec 27, 2023  3:48:00 PM",Aarush,why are you coming onto me,6,1,,False

"

In [13]:
imessage_df = pd.read_csv(path_csv_separate)

#drop duplicates
print(imessage_df.duplicated().sum())

imessage_df = imessage_df.drop_duplicates()



1


In [14]:
# Define the format string
date_format = '%b %d, %Y %I:%M:%S %p'

# Function to check if a date matches the format
def is_valid_date(date_str, format_str):
    try:
        pd.to_datetime(date_str, format=format_str)
        return True
    except ValueError:
        return False
# Apply the function and filter out invalid dates
imessage_df['is_valid'] = imessage_df['date'].apply(lambda x: is_valid_date(x, date_format))
imessage_df = imessage_df[imessage_df['is_valid']].drop(columns='is_valid')

# Convert the valid dates to the desired format
imessage_df['date'] = pd.to_datetime(imessage_df['date'], format=date_format)
imessage_df['formatted_date'] = imessage_df['date'].dt.strftime('%d/%m/%Y %H:%M:%S')

# Display the filtered DataFrame
print(imessage_df.head())

                 date      user  \
0 2023-12-27 15:30:14  Samyukta   
1 2023-12-27 15:30:36    Aarush   
2 2023-12-27 15:30:39    Aarush   
3 2023-12-27 15:34:07  Samyukta   
4 2023-12-27 15:34:15  Samyukta   

                                                text  word count  \
0  Hey girl! Big fan of your writing Xx- Collin H...          10   
1                                               stop           1   
2                                   send me location           3   
3                                                Why           1   
4    girl get on snapchat if u want to see locations          10   

   number of separated texts reactions  game_pigeon       formatted_date  
0                          1       NaN        False  27/12/2023 15:30:14  
1                          1       NaN        False  27/12/2023 15:30:36  
2                          1       NaN        False  27/12/2023 15:30:39  
3                          1       NaN        False  27/12/2023 15:34:07  
4    

In [15]:
imessage_df['platform'] = 'IMessage'

In [16]:
#export to csv file
imessage_df.to_csv(path_csv_separate)

In [17]:
imsg_df_grouped = imessage_df.copy()

In [18]:
imsg_df_grouped['text'] = imsg_df_grouped['text'].fillna('')
imsg_df_grouped['group'] = (imsg_df_grouped['user'] != imsg_df_grouped['user'].shift()).cumsum()
imsg_df_grouped['number of separated texts'] = imsg_df_grouped['group']
result_df = imsg_df_grouped.groupby(['user', 'group']).agg({
    'text': lambda x: '\n'.join(x),  # Combine texts with newline separator
    'date' : 'first',
    'word count' : 'sum',
    'user': 'size'  # Count number of rows merged
}).rename(columns={'user': 'number of separated texts'})


# Reset index to convert group columns back to regular columns
result_df = result_df.reset_index()
result_df = result_df.sort_values(by='date')
result_df = result_df.drop('group', axis = 1)
# Display the result

print(result_df.head())

          user                                               text  \
1172  Samyukta  Hey girl! Big fan of your writing Xx- Collin H...   
0       Aarush                             stop\nsend me location   
1173  Samyukta  Why\ngirl get on snapchat if u want to see loc...   
1       Aarush  can u stop being my opp\nattachments/11/dea07b...   
1174  Samyukta  Get off iMessage and get on snapchat boy!!\nSn...   

                    date  word count  number of separated texts  
1172 2023-12-27 15:30:14          10                          1  
0    2023-12-27 15:30:36           4                          2  
1173 2023-12-27 15:34:07          11                          2  
1    2023-12-27 15:34:38           7                          2  
1174 2023-12-27 15:47:28          15                          3  


In [19]:
def combine_consecutive_messages(df):
    # Create a column to mark groups of consecutive rows with the same user
    df['group'] = (df['user'] != df['user'].shift()).cumsum()

    def first_non_nan_reaction(reactions):
        # Filter out NaN values and get the first value if any
        non_nan_reactions = [r for r in reactions if pd.notna(r)]
        return non_nan_reactions[0] if non_nan_reactions else pd.NA
    
    # Group by 'user' and 'group', and aggregate the texts and count of messages
    result_df = df.groupby(['user', 'group']).agg({
        'text': lambda x: '\n'.join(x),  # Combine texts with newline separator
        'word count': 'sum',  # Sum of word counts for merged messages
        'date': 'first',  # Take the first date in the group
        'reactions': first_non_nan_reaction,  # Combine reactions into a list
        'game_pigeon' : 'first',
        'user' : 'size'
    }).rename(columns={'user': 'number of separated texts'})
    
    # Reset index to convert 'user' and 'group' to columns
    result_df = result_df.reset_index()
    
    # Drop the 'group' column as it is no longer needed
    result_df = result_df.drop(columns=['group'])
    
    # Sort by date if needed
    result_df = result_df.sort_values(by='date')
    
    return result_df

# Example usage:
imsg_df_grouped['text'] = imsg_df_grouped['text'].fillna('')
result_df = combine_consecutive_messages(imsg_df_grouped)

result_df['platform'] = 'IMessage'
# Display the result
print(result_df.head())


          user                                               text  word count  \
1172  Samyukta  Hey girl! Big fan of your writing Xx- Collin H...          10   
0       Aarush                             stop\nsend me location           4   
1173  Samyukta  Why\ngirl get on snapchat if u want to see loc...          11   
1       Aarush  can u stop being my opp\nattachments/11/dea07b...           7   
1174  Samyukta  Get off iMessage and get on snapchat boy!!\nSn...          15   

                    date reactions  game_pigeon  number of separated texts  \
1172 2023-12-27 15:30:14      <NA>        False                          1   
0    2023-12-27 15:30:36      <NA>        False                          2   
1173 2023-12-27 15:34:07      <NA>        False                          2   
1    2023-12-27 15:34:38      <NA>        False                          2   
1174 2023-12-27 15:47:28      <NA>        False                          3   

      platform  
1172  IMessage  
0     IMes

In [20]:
result_df.to_csv(path_csv_grouped)